In [ ]:
from io import StringIO

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import NaN
from functions import * # import functions from the .py file

from pylab import rcParams
rcParams["figure.figsize"] = 16, 4

PAYLOADSIZE = 14

if PAYLOADSIZE % 2 != 0:
    print("Alarm! the payload size is not even.")
NUM_16RND = (PAYLOADSIZE-2)//2 # how many 16 bits random number included in each frame
MAX_SEQ = 256 # (decimal) maximum seq number defined by the length of the seq, the length of seq is 1B

## Log file

**For a fair comparison between groups, must follow the output format below**
<br>$\color{red}{\text{timestamp_rx }|\text{ len [hex] } \text{ seq [hex] } \text{ payload [hex] }|\text{rssi}}$
<br>$\text{ len [hex]}$ = payload length + 1 (seq. field length [1B])
<br>$\text{ payload [hex]}$ should start with the pseudo sequence number [2B], indicates the starting byte's position in the file.
***
*Example experiment file output for one frame with reciever timestamp xxxx, frame length, seq number and random payload, received signal strength.*
<br>15:17:18.394 |0f 46 bd f8 f7 ea 41 ac 1b be 05 a0 7f d2 09 d4 |  -91 CRC error
<br>$\color{red}{\text{delimiter for four variables is "|" and delimiter for the payload each byte is the space " ".}}$

In [ ]:
import glob

clock = 125
experiments = pd.DataFrame()
experiments_df = dict()
logdir = "./log/"
for path in glob.glob(f"{logdir}*.txt"):
    filename = path.removeprefix(logdir)
    params = filename.removesuffix(".txt").split("_")
    data = {
        "filename": filename,
        "a": int(params[0]),
        "b": int(params[1]),
        "d1":int(params[2]), #cm
        "d2":int(params[3]), #cm
        "baudrate": int(params[4].replace('k', '000')),
        "receiver": params[5],
        "delta_fsk": clock/int(params[0]) - clock/int(params[1]),
        "file_delay_s": None,
        "bit_reliability": None,
        "dis_metric": None,
        "delta_f": (clock/int(params[0]) + clock/int(params[1]))/2,
    }
    experiments_df[filename] = readfile(path)
    experiments = pd.concat([pd.DataFrame(data, index=[0]), experiments], ignore_index=True)
print(experiments)


In [ ]:
''' Calculate the file delay, bit reliability, and distance metric for each experiment'''

for idx, experiment in experiments.iterrows():
    print(f"Processing {experiment['filename']}")
    df = experiments_df[experiment["filename"]]
    
    # delete the corrupted packets (PAYLOADSIZE + 2B pesudo sequence number)
    df = df[df.payload.apply(lambda x: len(x)==((PAYLOADSIZE)*3-1))]
    df.reset_index(inplace=True)
    # replace the resettable seq number to unresettable, for the purpose of reliability calculation
    test = replace_seq(df.copy(), MAX_SEQ=256)
    test.rename(columns = {'seq':'old_seq', 'new_seq':'seq'}, inplace = True)

    # compute the file delay
    file_delay = df.time_rx[len(df) - 1] - df.time_rx[0]
    # convert the timedelta to s
    file_delay_s = np.timedelta64(file_delay, "ms").astype(int) / 1000
    experiments.loc[idx, "file_delay_s"] = file_delay_s

    # compute the BER for all received packets
    # return the in total ber for received file, error statistics and correct file content supposed to be transmitted
    ber, error, file_content = compute_ber(test, PACKET_LEN=NUM_16RND*2, MAX_SEQ=MAX_SEQ)
    bit_reliability = (1-ber)*100
    experiments.loc[idx, "bit_reliability"] = bit_reliability

    # record the distance
    dis_carrier_tag = experiment["d1"]/100 # cm -> m
    dis_tag_rx = experiment["d2"]/100 # cm -> m

    dis_metric = dis_carrier_tag**2*dis_tag_rx**2
    experiments.loc[idx, "dis_metric"] = dis_metric


In [ ]:
fig, ax = plt.subplots()

# Plot the fig for each baudrate and distance
## filter out baudrate not equal to 90000
data = experiments.loc[experiments["baudrate"] == 90000]
for key, grp in data.groupby(['d1','d2','baudrate']):
    ## filter out the delta_fsk >= 1
    grp = grp.loc[grp["delta_fsk"] < 1]
    grp.sort_values(by=['delta_fsk'], inplace=True)
    ax = plt.plot(grp['delta_fsk'], grp['bit_reliability'], label=key, marker='o', linestyle='--', markersize=5)

plt.title('Bit Reliability vs. Delta_fsk (d1, d2, baudrate)')
plt.xlabel('delta_fsk')
plt.ylabel('bit_reliability')
plt.legend(title='(d1, d2, baudrate)')
plt.show()

In [ ]:
fig, ax = plt.subplots()

# Plot the fig for each baudrate and distance
## filter out baudrate not equal to 90000
data = experiments.loc[experiments["baudrate"] == 90000]
for key, grp in data.groupby(['d1','d2','baudrate']):
    ## filter out the delta_fsk >= 1
    grp = grp.loc[grp["delta_fsk"] < 1]
    grp.sort_values(by=['delta_fsk'], inplace=True)
    ax = plt.plot(grp['delta_fsk'], grp['file_delay_s'], label=key, marker='o', linestyle='--', markersize=5)

plt.title('Total 100 Packets Transmission Time vs. Delta_fsk (d1, d2, baudrate)')
plt.xlabel('delta_fsk')
plt.ylabel('time (second)')
plt.legend(title='(d1, d2, baudrate)')
plt.show()

# Radarplot

**Please pay attention to the unit**
<br> <font color='red'>Time</font>: use second as unit
<br> <font color='red'>Reliability</font>: use the percent as unit
<br> <font color='red'>Distance</font>: use meter as unit
<br>**Metics = [Time(s), Reliability(%), Distance(m)]**
<br> <font color='red'>Always keep the reference in your plot.</font>
<br> Tips: for ploting, time metric uses $\frac{1}{Time}*1000$ as the time metric, the higher the value the better the system performance

In [ ]:
for idx, experiment in experiments.iterrows():
    metrics = [experiment["file_delay_s"], experiment["bit_reliability"], experiment["dis_metric"]]
    radar_plot(metrics)